<a href="https://colab.research.google.com/github/csaatechnicalarts/ML_Bootcamp/blob/main/DnD5e_Monsters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Analysis of D&D 5th Edition Monsters (Kaggle Dataset)

##Content Outline
* Loading the Data
* Data Reorganization and Cleanup

In [140]:
import pandas as pd

##Loading the Data
Kaggle has a handy [dataset for D&D 5th Edition monsters](https://www.kaggle.com/datasets/patrickgomes/dungeons-and-dragons-5e-monsters). For our work we download the CSV file, rename it as "DnD5e_monsters_orig.csv" and upload it to Colab.

In [141]:
df = pd.read_csv('sample_data/DnD5e_monsters_orig.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 324 entries, 0 to 323
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Name                    324 non-null    object
 1   Size                    324 non-null    object
 2   Race + alignment        324 non-null    object
 3   HP                      324 non-null    object
 4   Armor                   324 non-null    object
 5   Speed                   324 non-null    object
 6   Challenge rating  (XP)  324 non-null    object
dtypes: object(7)
memory usage: 17.8+ KB


In [142]:
df.head()

,Name,Size,Race + alignment,HP,Armor,Speed,Challenge rating (XP)
0,Aboleth,Large,"aberration, Lawful Evil",135 (18d10+36),17 (Natural Armor),"10 ft., swim 40 ft.","10 (5,900 XP)"
1,Acolyte,Medium,"humanoid (any race), Any Alignment",9 (2d8),10,30 ft.,1/4 (50 XP)
2,Adult Black Dragon,Huge,"dragon, Chaotic Evil",195 (17d12+85),19 (Natural Armor),"40 ft., fly 80 ft., swim 40 ft.","14 (11,500 XP)"
3,Adult Blue Dragon,Huge,"dragon, Lawful Evil",225 (18d12+108),19,"40 ft., burrow 30 ft., fly 80 ft.","16 (15,000 XP)"
4,Adult Brass Dragon,Huge,"dragon, Chaotic Good",172 (15d12+75),18 (Natural Armor),"40 ft., burrow 40 ft., fly 80 ft.","13 (10,000 XP)"


##Data Reorganization and Cleanup
The data set has eight columns and 324 rows. Looking at the column headers, we'd like to spli the **Race + alignment** information into two separate pieces of information. So the first alteration we perform is to split that column.

In [143]:
split_racealignment_df = df.copy()
split_race_alignment_df = split_racealignment_df['Race + alignment'].str.split(',', expand=True)
split_racealignment_df['Race'] = split_race_alignment_df[0]
split_racealignment_df['Alignment'] = split_race_alignment_df[1]
split_racealignment_df.drop(columns=['Race + alignment'], inplace=True)
split_racealignment_df.head()

,Name,Size,HP,Armor,Speed,Challenge rating (XP),Race,Alignment
0,Aboleth,Large,135 (18d10+36),17 (Natural Armor),"10 ft., swim 40 ft.","10 (5,900 XP)",aberration,Lawful Evil
1,Acolyte,Medium,9 (2d8),10,30 ft.,1/4 (50 XP),humanoid (any race),Any Alignment
2,Adult Black Dragon,Huge,195 (17d12+85),19 (Natural Armor),"40 ft., fly 80 ft., swim 40 ft.","14 (11,500 XP)",dragon,Chaotic Evil
3,Adult Blue Dragon,Huge,225 (18d12+108),19,"40 ft., burrow 30 ft., fly 80 ft.","16 (15,000 XP)",dragon,Lawful Evil
4,Adult Brass Dragon,Huge,172 (15d12+75),18 (Natural Armor),"40 ft., burrow 40 ft., fly 80 ft.","13 (10,000 XP)",dragon,Chaotic Good


Problems immediately arise with our splitting of the **Race + alginment** column along the comma character **(,)**. An extraneous comma chacter is embedded in the column for 'Imp' and the various were-monsters ('Werebear', 'Wereboar', 'Wererat', 'Weretiger', 'Werewolf'). Faced with these, our Pandas *str.split()* operation will not separate the race data from the alignment data correctly as there are two comma characters it has to deal with.

Here are the offending rows.

In [144]:
import re
bad_racealign_df = df[df['Race + alignment'].apply(lambda x: True if re.search('.*,.*,', x) else False)]
bad_racealign_df

,Name,Size,Race + alignment,HP,Armor,Speed,Challenge rating (XP)
173,Imp,Tiny,"fiend (devil, shapechanger), Lawful Evil",10 (3d4+3),13,"20 ft., fly 40 ft.",1 (200 XP)
299,Werebear,Medium,"humanoid (human, shapechanger), Neutral Good",135 (18d8+54),"10 (In Humanoid Form, 11 In Bear And Hybrid Fo...","30 ft. (40 ft., climb 30 ft. in bear or hybrid...","5 (1,800 XP)"
300,Wereboar,Medium,"humanoid (human, shapechanger), Neutral Evil",78 (12d8+24),"10 (In Humanoid Form, 11 In Boar And Hybrid Fo...",30 ft. (40 ft. in boar form),"4 (1,100 XP)"
301,Wererat,Medium,"humanoid (human, shapechanger), Lawful Evil",33 (6d8+6),12,30 ft.,2 (450 XP)
302,Weretiger,Medium,"humanoid (human, shapechanger), Neutral",120 (16d8+48),12,30 ft. (40 ft. in tiger form),"4 (1,100 XP)"
303,Werewolf,Medium,"humanoid (human, shapechanger), Chaotic Evil",58 (9d8+18),"11 (In Humanoid Form, 12 In Wolf Or Hybrid Form)",30 ft. (40 ft. in wolf form),3 (700 XP)


To deal with this obstacle, let's get the list of indices of the offending rows.

In [145]:
bad_racealign_idx = list(bad_racealign_df.index)
bad_racealign_idx

[173, 299, 300, 301, 302, 303]

Then it's a simple matter up applying a lambda that uses regex to subsitute the first comma with a space, updating only the rows in out **bad_realign_idx** list.

In [146]:
df.loc[bad_racealign_idx, 'Race + alignment'] = df.loc[bad_racealign_idx, 'Race + alignment'].apply(lambda str: re.sub(',', ' ', str, count=1))
df.loc[bad_racealign_idx]

,Name,Size,Race + alignment,HP,Armor,Speed,Challenge rating (XP)
173,Imp,Tiny,"fiend (devil shapechanger), Lawful Evil",10 (3d4+3),13,"20 ft., fly 40 ft.",1 (200 XP)
299,Werebear,Medium,"humanoid (human shapechanger), Neutral Good",135 (18d8+54),"10 (In Humanoid Form, 11 In Bear And Hybrid Fo...","30 ft. (40 ft., climb 30 ft. in bear or hybrid...","5 (1,800 XP)"
300,Wereboar,Medium,"humanoid (human shapechanger), Neutral Evil",78 (12d8+24),"10 (In Humanoid Form, 11 In Boar And Hybrid Fo...",30 ft. (40 ft. in boar form),"4 (1,100 XP)"
301,Wererat,Medium,"humanoid (human shapechanger), Lawful Evil",33 (6d8+6),12,30 ft.,2 (450 XP)
302,Weretiger,Medium,"humanoid (human shapechanger), Neutral",120 (16d8+48),12,30 ft. (40 ft. in tiger form),"4 (1,100 XP)"
303,Werewolf,Medium,"humanoid (human shapechanger), Chaotic Evil",58 (9d8+18),"11 (In Humanoid Form, 12 In Wolf Or Hybrid Form)",30 ft. (40 ft. in wolf form),3 (700 XP)


Now we can go ahead and split the **Race + alignment** column into two, removing the original column in the process.

In [147]:
split_racealignment_df = df.copy()
split_columns_df = copy_df['Race + alignment'].str.split(',', expand=True)
split_racealignment_df['Race'] = split_columns_df[0]
split_racealignment_df['Alignment'] = split_columns_df[1]
split_racealignment_df.drop(columns=['Race + alignment'], inplace=True)

In [148]:
split_racealignment_df.groupby('Race').count()

,Name,Size,HP,Armor,Speed,Challenge rating (XP),Alignment
Race,,,,,,,
Fiend (Shapechanger),2,2,2,2,2,2,2
aberration,5,5,5,5,5,5,5
beast,87,87,87,87,87,87,87
celestial,6,6,6,6,6,6,6
construct,9,9,9,9,9,9,9
dragon,43,43,43,43,43,43,43
elemental,16,16,16,16,16,16,16
fey,6,6,6,6,6,6,6
fiend,4,4,4,4,4,4,4


Now we come to the second update that we need to perform. Scanning that output, we notice there are two entries that overlap:
*   Fiend (Shapechanger)
*   fiend (shapechanger)

In fact, examining the split data frame, we isolate the three rows.



In [149]:
split_racealignment_df.loc[split_racealignment_df['Race'].isin(['Fiend (Shapechanger)','fiend (shapechanger)'])]

,Name,Size,HP,Armor,Speed,Challenge rating (XP),Race,Alignment
174,Incubus,Medium,66 (12d8 + 12),15 (Natural Armor),"30 ft., fly 60 ft.","4 (1,100 XP)",Fiend (Shapechanger),Neutral Evil
268,Succubus,Medium,66 (12d8 + 12),15 (Natural Armor),"30 ft., fly 60 ft.","4 (1,100 XP)",Fiend (Shapechanger),Neutral Evil
269,Succubus (Incubus),Medium,66 (12d8+12),15 (Natural Armor),"30 ft., fly 60 ft.","4 (1,100 XP)",fiend (shapechanger),Neutral Evil


It's clear that index 269 duplicates the information in rows 174 and 268. We go ahead then and delete row 269.

In [150]:
split_racealignment_df.drop(index=269, inplace=True)

We've reached a good checkpoint in our work. Before we proceed, lets store the cleaned-up data.

In [152]:
split_racealignment_df.to_csv('sample_data/DnD5e_monsters_cleaned.csv')